# EDA – BikePoA (Porto Alegre)

Este notebook explora snapshots do GBFS e clima (Open-Meteo) armazenados em SQLite.

Requisitos: `pip install -r requirements.txt`.

In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path

DB = Path('data/bikepoa.sqlite')
conn = sqlite3.connect(DB)


In [ ]:
# Resumo da rede no último snapshot
q = '''
WITH last_scrape AS (
  SELECT MAX(scraped_at) AS ts FROM station_status
)
SELECT
  COUNT(DISTINCT s.station_id) AS estacoes,
  SUM(COALESCE(s.capacity,0)) AS capacidade_total,
  SUM(ss.num_bikes_available) AS bikes_disp,
  SUM(ss.num_docks_available) AS docks_disp
FROM station_status ss
JOIN last_scrape ls ON ss.scraped_at = ls.ts
JOIN stations s ON s.station_id = ss.station_id;
'''
df = pd.read_sql(q, conn)
df

In [ ]:
# Top estações por ocupação
q = Path('sql/queries.sql').read_text(encoding='utf-8').split(';')
print(q[1] + ';')
pd.read_sql(q[1] + ';', conn).head(10)

In [ ]:
# Série horária média e junção com clima
q = Path('sql/queries.sql').read_text(encoding='utf-8')
q